In [1]:
import dask_awkward as dak
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import awkward as ak
import dask_awkward as dak
import numpy as np
import json
import argparse
import os
from distributed import Client
import time    
import tqdm
import mplhep as hep
import matplotlib.pyplot as plt
import matplotlib
import glob
hep.style.use("CMS")
from dask_gateway import Gateway
# gateway = Gateway(
#     "http://dask-gateway-k8s.geddes.rcac.purdue.edu/",
#     proxy_address="traefik-dask-gateway-k8s.cms.geddes.rcac.purdue.edu:8786",
# )
# cluster_info = gateway.list_clusters()[0]# get the first cluster by default. There only should be one anyways
# client = gateway.connect(cluster_info.name).get_client()
# print("Gateway Client created")
client =  Client(n_workers=30,  threads_per_worker=1, processes=True, memory_limit='10 GiB') 

2024-05-10 19:25:14,940 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:40293' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('zip-957bb4e6bcb5e199c53ac0990524e5af', 846), ('zip-957bb4e6bcb5e199c53ac0990524e5af', 6953), ('zip-957bb4e6bcb5e199c53ac0990524e5af', 871), ('zip-957bb4e6bcb5e199c53ac0990524e5af', 817), ('zip-957bb4e6bcb5e199c53ac0990524e5af', 736), ('zip-957bb4e6bcb5e199c53ac0990524e5af', 710), ('zip-957bb4e6bcb5e199c53ac0990524e5af', 7055), ('zip-957bb4e6bcb5e199c53ac0990524e5af', 925), ('zip-957bb4e6bcb5e199c53ac0990524e5af', 7026), ('zip-957bb4e6bcb5e199c53ac0990524e5af', 6901), ('zip-957bb4e6bcb5e199c53ac0990524e5af', 899), ('zip-957bb4e6bcb5e199c53ac0990524e5af', 7003), ('zip-957bb4e6bcb5e199c53ac0990524e5af', 6974), ('zip-957bb4e6bcb5e199c53ac0990524e5af', 760), ('zip-957bb4e6bcb5e199c53ac0990524e5af', 98), ('zip-957bb4e6bcb5e199c53ac0990524e5af', 792), ('zip-957bb4e6bcb5e199c53ac0990524e5af', 952)} (stimu

In [2]:
# get data events
common_load_path = "/depot/cms/users/yun79/results/stage1/Run2StorageTest/2018/f1_0"
# common_load_path = "/work/users/yun79/stage1_output/Run2StorageTest/2018/f1_0"
# data_load_path = common_load_path+"/data*/*/*.parquet"
# data_load_path = common_load_path+"/data_C/*/*.parquet"
data_load_path = common_load_path+"/data_D/*/*.parquet"

data_events = dak.from_parquet(data_load_path) 
# we're only interested in ZCR
filter = ak.fill_none(data_events["z_peak"], value=False)
data_events = data_events[filter]



In [4]:
# fields_of_interest = ["mu1_pt", "mu1_eta", "mu2_eta","dimuon_mass", "weights", "fraction"]
fields_of_interest = ["mu1_pt", "mu1_eta", "mu2_eta","dimuon_mass", "fraction","weights"]

# events = mc_events[fields_of_interest].compute() # eager version
test_events = data_events[fields_of_interest]
test_events

dask.awkward<getitem, npartitions=5138>

In [5]:
test_events.fields

['mu1_pt', 'mu1_eta', 'mu2_eta', 'dimuon_mass', 'fraction', 'weights']

In [ ]:

zip = ak.zip({
    field : test_events[field] for field in test_events.fields
})
zip.compute()

In [3]:
# get MC events (all MC except DY)
secondary_MC_strs = [
    "ttjet*", # all TT samples
    "st_tw*", # single Top samples
    "ewk*", # single Top samples
    "ww*", # diboson samples
    "wz*", # diboson samples
    "zz*", # diboson samples
]
filelist_big = []
for mc_str in secondary_MC_strs:
    filelist = glob.glob(common_load_path + f"/{mc_str}/*/*.parquet")
    filelist_big += filelist
mc_events = dak.from_parquet(filelist_big) 
# we're only interested in ZCR
filter = ak.fill_none(mc_events["z_peak"], value=False)
mc_events = mc_events[filter]

In [4]:
def get_calib_categories(events):
    BB = ((abs(events["mu1_eta"])<=0.9) & (abs(events["mu2_eta"])<=0.9))
    BO = ((abs(events["mu1_eta"])<=0.9) & ((abs(events["mu2_eta"])>0.9) & (abs(events["mu2_eta"]) <=1.8)))
    BE = ((abs(events["mu1_eta"])<=0.9) & ((abs(events["mu2_eta"])>1.8) & (abs(events["mu2_eta"]) <=2.4)))
    OB = (((abs(events["mu1_eta"])>0.9) & (abs(events["mu1_eta"]) <=1.8)) & (abs(events["mu2_eta"])<=0.9))
    OO = (((abs(events["mu1_eta"])>0.9) & (abs(events["mu1_eta"]) <=1.8)) & ((abs(events["mu2_eta"])>0.9) & (abs(events["mu2_eta"]) <=1.8)))
    OE = (((abs(events["mu1_eta"])>0.9) & (abs(events["mu1_eta"]) <=1.8)) & ((abs(events["mu2_eta"])>1.8) & (abs(events["mu2_eta"]) <=2.4)))
    EB = (((abs(events["mu1_eta"])>1.8) & (abs(events["mu1_eta"]) <=2.4)) & (abs(events["mu2_eta"])<=0.9))
    EO = (((abs(events["mu1_eta"])>1.8) & (abs(events["mu1_eta"]) <=2.4)) & ((abs(events["mu2_eta"])>0.9) & (abs(events["mu2_eta"]) <=1.8)))
    EE = (((abs(events["mu1_eta"])>1.8) & (abs(events["mu1_eta"]) <=2.4)) & ((abs(events["mu2_eta"])>1.8) & (abs(events["mu2_eta"]) <=2.4)))
    categories = [((events["mu1_pt"]>30)&(events["mu1_pt"]<=45)&(BB | OB | EB)),
                          ((events["mu1_pt"]>30)&(events["mu1_pt"]<=45)&(BO | OO | EO)),
                          ((events["mu1_pt"]>30)&(events["mu1_pt"]<=45)&(BE | OE | EE)),
                          ((events["mu1_pt"]>45)&(events["mu1_pt"]<=52)&BB),
                          ((events["mu1_pt"]>45)&(events["mu1_pt"]<=52)&BO),
                          ((events["mu1_pt"]>45)&(events["mu1_pt"]<=52)&BE),
                          ((events["mu1_pt"]>45)&(events["mu1_pt"]<=52)&OB),
                          ((events["mu1_pt"]>45)&(events["mu1_pt"]<=52)&OO),
                          ((events["mu1_pt"]>45)&(events["mu1_pt"]<=52)&OE),
                          ((events["mu1_pt"]>45)&(events["mu1_pt"]<=52)&EB),
                          ((events["mu1_pt"]>45)&(events["mu1_pt"]<=52)&EO),
                          ((events["mu1_pt"]>45)&(events["mu1_pt"]<=52)&EE),
                  # voigtian start here onwards
                          ((events["mu1_pt"]>52)&(events["mu1_pt"]<=62)&BB),
                          ((events["mu1_pt"]>52)&(events["mu1_pt"]<=62)&BO),
                          ((events["mu1_pt"]>52)&(events["mu1_pt"]<=62)&BE),
                          ((events["mu1_pt"]>52)&(events["mu1_pt"]<=62)&OB),
                          ((events["mu1_pt"]>52)&(events["mu1_pt"]<=62)&OO),
                          ((events["mu1_pt"]>52)&(events["mu1_pt"]<=62)&OE),
                          ((events["mu1_pt"]>52)&(events["mu1_pt"]<=62)&EB),
                          ((events["mu1_pt"]>52)&(events["mu1_pt"]<=62)&EO),
                          ((events["mu1_pt"]>52)&(events["mu1_pt"]<=62)&EE),
                          ((events["mu1_pt"]>62)&(events["mu1_pt"]<=200)&BB),
                          ((events["mu1_pt"]>62)&(events["mu1_pt"]<=200)&BO),
                          ((events["mu1_pt"]>62)&(events["mu1_pt"]<=200)&BE),
                          ((events["mu1_pt"]>62)&(events["mu1_pt"]<=200)&OB),
                          ((events["mu1_pt"]>62)&(events["mu1_pt"]<=200)&OO),
                          ((events["mu1_pt"]>62)&(events["mu1_pt"]<=200)&OE),
                          ((events["mu1_pt"]>62)&(events["mu1_pt"]<=200)&EB),
                          ((events["mu1_pt"]>62)&(events["mu1_pt"]<=200)&EO),
                          ((events["mu1_pt"]>62)&(events["mu1_pt"]<=200)&EE),]
    return categories

In [11]:

# fields_of_interest = ["mu1_pt", "mu1_eta", "mu2_eta","dimuon_mass", "weights", "fraction"]
# # events = mc_events[fields_of_interest].compute() # eager version
# events = data_events[fields_of_interest]
# events

In [12]:
"""
Eager load variable all at once Test
"""
import time
time_step = time.time()
binning = np.linspace(*[70, 110, 100])
# fraction_weight = ak.to_numpy(1/events.fraction.compute())
fields_of_interest = ["mu1_pt", "mu1_eta", "mu2_eta","dimuon_mass", "weights", "fraction"]
data_events = data_events[fields_of_interest].compute() # get eager version
mc_events = mc_events[fields_of_interest].compute() # get eager version
data_categories = get_calib_categories(data_events)
mc_categories = get_calib_categories(mc_events)
for idx in range(len(data_categories)):
    cat_selection = data_categories[idx]
    # print(f"cat_selection: {cat_selection.compute()}")
    fraction_weight = 1/data_events.fraction
    weights = data_events.weights*fraction_weight*cat_selection
    # weights = ak.to_numpy(weights.compute())
    dimuon_mass = ak.fill_none(data_events.dimuon_mass, value=-999.0)
    # dimuon_mass = ak.to_numpy(dimuon_mass.compute())
    filter = (dimuon_mass!=-999.0) & (weights != 0)
    dimuon_mass = dimuon_mass[filter]
    weights = weights[filter]
    data_hist, _ = np.histogram(dimuon_mass, bins=binning, weights = weights)
    # MC ------------------------------------------------------------------------------------
    cat_selection = mc_categories[idx]
    # print(f"cat_selection: {cat_selection.compute()}")
    fraction_weight = 1/mc_events.fraction
    weights = mc_events.weights*fraction_weight*cat_selection
    # weights = ak.to_numpy(weights.compute())
    dimuon_mass = ak.fill_none(mc_events.dimuon_mass, value=-999.0)
    # dimuon_mass = ak.to_numpy(dimuon_mass.compute())
    filter = (dimuon_mass!=-999.0) & (weights != 0)
    dimuon_mass = dimuon_mass[filter]
    weights = weights[filter]
    mc_hist, _ = np.histogram(dimuon_mass, bins=binning, weights = weights)
    

    fig, ax_main = plt.subplots()
    hep.histplot(
        data_hist, xerr=True, 
        # yerr=data_err,
        bins=binning, stack=False, histtype='errorbar', color='black', 
        label='Data', ax=ax_main
    )
    hep.histplot(mc_hist, bins=binning, 
                             stack=False, histtype='fill', 
                             label="MC", 
                             # sort='label_r', 
                             color="Green", 
                             ax=ax_main)
    ax_main.legend(loc="upper right")
    ax_main.set_xlabel("dimuon mass [GeV]")
    ax_main.set_title(f"Cat {idx} ZCR mass distribution")
    plt.savefig(f"./cat{idx}.pdf")
    ax_main.set_yscale('log')
    ax_main.set_ylim(0.01, 1e9)
    plt.savefig(f"./cat{idx}_log.pdf")
    print(f"time elapsed in : {time.time()-time_step}")
    time_step = time.time()
    plt.clf()



time elapsed in : 135.25930619239807
time elapsed in : 3.6744840145111084
time elapsed in : 3.4598801136016846
time elapsed in : 3.672902822494507
time elapsed in : 3.2747693061828613
time elapsed in : 3.227245807647705
time elapsed in : 3.444131374359131
time elapsed in : 3.3161158561706543
time elapsed in : 3.0586585998535156
time elapsed in : 3.00970458984375
time elapsed in : 3.205486297607422
time elapsed in : 3.075040102005005
time elapsed in : 3.2487900257110596
time elapsed in : 3.1352314949035645
time elapsed in : 3.3327343463897705
time elapsed in : 3.1848297119140625
time elapsed in : 3.1109938621520996


KeyboardInterrupt: 

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

...

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
2024-05-10 19:01:03,459 - distributed.nanny - ERROR - Worker process died unexpectedly
2024-05-10 19:01:03,460 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
2024-05-10 19:01:03,460 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
2024-05-10 19:01:03,461 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
Traceback (most recent call last):
  File "/depot/cms/kernels/coffea2024/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/depot/cms/kernels/coffea2024/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/depot/cms/kernels/coffea2024/lib/python3.10/site-packages/distributed/process.py", line 20

In [5]:
"""
Eager load variable all at once Zip Test
"""
import time
time_step = time.time()
binning = np.linspace(*[70, 110, 100])
# fraction_weight = ak.to_numpy(1/events.fraction.compute())
fields_of_interest = ["mu1_pt", "mu1_eta", "mu2_eta","dimuon_mass", "weights", "fraction"]
data_events = data_events[fields_of_interest]
data_events = ak.zip({
    field : data_events[field] for field in data_events.fields
}).compute()
mc_events = mc_events[fields_of_interest]
mc_events = ak.zip({
    field : mc_events[field] for field in mc_events.fields
}).compute()
data_categories = get_calib_categories(data_events)
mc_categories = get_calib_categories(mc_events)
for idx in range(len(data_categories)):
    cat_selection = data_categories[idx]
    # print(f"cat_selection: {cat_selection.compute()}")
    fraction_weight = 1/data_events.fraction
    weights = data_events.weights*fraction_weight*cat_selection
    # weights = ak.to_numpy(weights.compute())
    dimuon_mass = ak.fill_none(data_events.dimuon_mass, value=-999.0)
    # dimuon_mass = ak.to_numpy(dimuon_mass.compute())
    filter = (dimuon_mass!=-999.0) & (weights != 0)
    dimuon_mass = dimuon_mass[filter]
    weights = weights[filter]
    data_hist, _ = np.histogram(dimuon_mass, bins=binning, weights = weights)
    # MC ------------------------------------------------------------------------------------
    cat_selection = mc_categories[idx]
    # print(f"cat_selection: {cat_selection.compute()}")
    fraction_weight = 1/mc_events.fraction
    weights = mc_events.weights*fraction_weight*cat_selection
    # weights = ak.to_numpy(weights.compute())
    dimuon_mass = ak.fill_none(mc_events.dimuon_mass, value=-999.0)
    # dimuon_mass = ak.to_numpy(dimuon_mass.compute())
    filter = (dimuon_mass!=-999.0) & (weights != 0)
    dimuon_mass = dimuon_mass[filter]
    weights = weights[filter]
    mc_hist, _ = np.histogram(dimuon_mass, bins=binning, weights = weights)
    

    fig, ax_main = plt.subplots()
    hep.histplot(
        data_hist, xerr=True, 
        # yerr=data_err,
        bins=binning, stack=False, histtype='errorbar', color='black', 
        label='Data', ax=ax_main
    )
    hep.histplot(mc_hist, bins=binning, 
                             stack=False, histtype='fill', 
                             label="MC", 
                             # sort='label_r', 
                             color="Green", 
                             ax=ax_main)
    ax_main.legend(loc="upper right")
    ax_main.set_xlabel("dimuon mass [GeV]")
    ax_main.set_title(f"Cat {idx} ZCR mass distribution")
    plt.savefig(f"./cat{idx}.pdf")
    ax_main.set_yscale('log')
    ax_main.set_ylim(0.01, 1e9)
    plt.savefig(f"./cat{idx}_log.pdf")
    print(f"time elapsed in : {time.time()-time_step}")
    time_step = time.time()
    plt.clf()



time elapsed in : 112.4340353012085
time elapsed in : 3.570920705795288
time elapsed in : 3.530517101287842
time elapsed in : 3.4827396869659424
time elapsed in : 3.0998494625091553
time elapsed in : 2.9365060329437256
time elapsed in : 3.102440595626831
time elapsed in : 3.1333203315734863
time elapsed in : 2.9441370964050293
time elapsed in : 3.169881582260132
time elapsed in : 2.971480131149292
time elapsed in : 2.9544475078582764
time elapsed in : 3.119187593460083
time elapsed in : 2.9520514011383057
time elapsed in : 2.9359512329101562
time elapsed in : 2.9557509422302246
time elapsed in : 3.2264773845672607
time elapsed in : 2.8725831508636475
time elapsed in : 2.89878511428833
time elapsed in : 2.909139633178711


/tmp/ipykernel_672380/2063539259.py:45: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax_main = plt.subplots()


time elapsed in : 2.8913564682006836
time elapsed in : 3.2132999897003174
time elapsed in : 2.967803716659546
time elapsed in : 3.0582010746002197
time elapsed in : 3.0254459381103516
time elapsed in : 3.0322484970092773
time elapsed in : 2.903773069381714
time elapsed in : 2.8268661499023438
time elapsed in : 2.9510347843170166
time elapsed in : 2.8930742740631104


<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

...

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

In [5]:
"""
Baseline run setup
"""
import time
time_step = time.time()
binning = np.linspace(*[70, 110, 100])
# fraction_weight = ak.to_numpy(1/events.fraction.compute())
data_categories = get_calib_categories(data_events)
mc_categories = get_calib_categories(mc_events)
for idx in range(len(data_categories)):
    cat_selection = data_categories[idx]
    # print(f"cat_selection: {cat_selection.compute()}")
    fraction_weight = 1/data_events.fraction
    weights = data_events.weights*fraction_weight*cat_selection
    # weights = ak.to_numpy(weights.compute())
    dimuon_mass = ak.fill_none(data_events.dimuon_mass, value=-999.0)
    # dimuon_mass = ak.to_numpy(dimuon_mass.compute())
    filter = (dimuon_mass!=-999.0) & (weights != 0)
    dimuon_mass = dimuon_mass[filter]
    weights = weights[filter]
    # data_hist, _ = np.histogram(dimuon_mass.compute(), bins=binning, weights = weights.compute())
    zip = ak.zip({
        "dimuon_mass" : dimuon_mass,
        "weights" : weights,
    })
    zip = zip.compute()
    data_hist, _ = np.histogram(zip.dimuon_mass, bins=binning, weights = zip.weights)
    # MC ------------------------------------------------------------------------------------
    cat_selection = mc_categories[idx]
    # print(f"cat_selection: {cat_selection.compute()}")
    fraction_weight = 1/mc_events.fraction
    weights = mc_events.weights*fraction_weight*cat_selection
    # weights = ak.to_numpy(weights.compute())
    dimuon_mass = ak.fill_none(mc_events.dimuon_mass, value=-999.0)
    # dimuon_mass = ak.to_numpy(dimuon_mass.compute())
    filter = (dimuon_mass!=-999.0) & (weights != 0)
    dimuon_mass = dimuon_mass[filter]
    weights = weights[filter]
    # mc_hist, _ = np.histogram(dimuon_mass.compute(), bins=binning, weights = weights.compute())
    zip = ak.zip({
        "dimuon_mass" : dimuon_mass,
        "weights" : weights,
    })
    zip = zip.compute()
    mc_hist, _ = np.histogram(zip.dimuon_mass, bins=binning, weights = zip.weights)

    fig, ax_main = plt.subplots()
    hep.histplot(
        data_hist, xerr=True, 
        # yerr=data_err,
        bins=binning, stack=False, histtype='errorbar', color='black', 
        label='Data', ax=ax_main
    )
    hep.histplot(mc_hist, bins=binning, 
                             stack=False, histtype='fill', 
                             label="MC", 
                             # sort='label_r', 
                             color="Green", 
                             ax=ax_main)
    ax_main.legend(loc="upper right")
    ax_main.set_xlabel("dimuon mass [GeV]")
    ax_main.set_title(f"Cat {idx} ZCR mass distribution")
    plt.savefig(f"./cat{idx}.pdf")
    ax_main.set_yscale('log')
    ax_main.set_ylim(0.01, 1e9)
    plt.savefig(f"./cat{idx}_log.pdf")
    print(f"time elapsed in : {time.time()-time_step}")
    time_step = time.time()
    plt.clf()


time elapsed in : 117.05133008956909
time elapsed in : 114.86696529388428
time elapsed in : 114.66685771942139
time elapsed in : 103.26324391365051
time elapsed in : 103.58142328262329
time elapsed in : 104.91327786445618


2024-05-10 19:25:14,742 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('zip-957bb4e6bcb5e199c53ac0990524e5af', 6861))" coro=<Worker.execute() done, defined at /depot/cms/kernels/coffea2024/lib/python3.10/site-packages/distributed/worker_state_machine.py:3615>> ended with CancelledError
2024-05-10 19:25:15,097 - distributed.worker - WARNING - Scheduler was unaware of this worker; shutting down.
2024-05-10 19:25:15,099 - distributed.worker - WARNING - Scheduler was unaware of this worker; shutting down.
2024-05-10 19:25:15,100 - distributed.worker - WARNING - Scheduler was unaware of this worker; shutting down.
2024-05-10 19:25:15,100 - distributed.worker - WARNING - Scheduler was unaware of this worker; shutting down.
2024-05-10 19:25:15,102 - distributed.worker - WARNING - Scheduler was unaware of this worker; shutting down.
2024-05-10 19:25:15,103 - distributed.worker - WARNING - Scheduler was unaware of this worker; shutting down.


KeyboardInterrupt: 

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>